In [1]:
import numpy as np
import pandas as pd

In [15]:
"""
Objective: Create a function that provides top-k most similar songs given answers to QnA.

Input:
clusters: N x C matrix, with N = number of songs in dataset, C = number of clusters provided
weights: C length vector determining intial weight distribution of clusters -> default is 1/n
L: learning rate
M: momentum parameter
k: Number of songs to output
tol: tolerance (when to stop asking questions)
initial_songs: song that participant feels like vibing to 

Output: 
songs: k-length vector of song indices
song_weights: k-length vector of song weights

""" 

## Todo: Momentum-based option?

"""
Pseudocode:

old_weights = np.zeros(weights.shape)
intial_song = (minimize SSE of average of initial_songs)
initial_diff = 1 # select song most different
momentum = 0
initial_cluster_weights = [[1/num_clusters]
while SSE of (weights - old_weights) > tol: # our beliefs on weights are still changing
    # Select song comparison
    for song in songs:
        measure weighted indicator of whether song and initial_song are in the same cluster
    order songs by indicator, select the initial_diff-th percentile song
    input(initial_song, song)
    score = ind(song and )
    momentum = M * momentum + L * 

input?

"""

def top_k(clusters, weights, initial_song, song_names, artist_names, initial_diff = 5, L = 3, M = 0.5, k = 10, tol = 1e-3):
    N = clusters.shape[0]
    song_weights = np.ones(clusters.shape) / N
    #old_song_weights = np.zeros(clusters.shape)
    #initial_song = 0 #assume its always song 0 for now
    momentum = 0
    qn_num = 0
    while(initial_diff > tol):
        qn_num += 1 
        #print( initial_diff, momentum, initial_song)
        cluster_diffs = clusters - clusters[initial_song, :] # N x C 
        cluster_diffs_ones = np.ones(clusters.shape)
        cluster_diffs_ones[cluster_diffs == 0] = 0
        # Cluster_diffs_ones is a N x C matrix, x_ij is 1 if song i is in the same cluster
        # as initial_song in clustering set j, and 0 otherwise
        weighted_songs = np.mean(cluster_diffs_ones * weights.reshape(1, -1), axis = 1) 
        # N-length vector - weighted ind(songs are NOT in same cluster)
        #print(weighted_songs)
        # Find song to compare 
        other_song = np.where(weighted_songs == np.percentile(weighted_songs, initial_diff, interpolation = 'nearest'))
        #print(other_song, other_song[0], other_song[0][0])
        other_song = other_song[0]
        other_song = np.random.choice(other_song)
        if other_song == initial_song: break
        val = input(f"""Do you prefer {song_names[initial_song]} by {artist_names[initial_song]} or {song_names[other_song]} by {artist_names[other_song]}? 
    Choose on a scale of 1-10, with 1 being in favor of the first song, and 10 being in favor of the other.""")
        print(f'Question Number: {qn_num}, Percentile: {initial_diff}, M: {momentum}, value selected: {val}')
        print(song_names[initial_song], artist_names[initial_song], ",", song_names[other_song], artist_names[other_song])
        val = int(val)
        # Update momentum (this should make max change of initial_diff around 8)
        momentum = M * momentum + L * (5.5 - val) # if val > 5.5, then we should slow down momentum 
        #print(momentum)

        # Update weights
        weight_comp = np.abs(clusters[initial_song, :] - clusters[other_song, :])
        #print('test', clusters[initial_song, : ])
        #print('test2', clusters[other_song, : ])
        #print(weight_comp)
        weight_comp_2 = np.ones(clusters.shape[1])
        # when diff > 50, penalize same cluster, when diff < 50, reward same cluster
        if initial_diff > 50:
            weight_comp_2 *= (1 - momentum / 100)
        else:
            weight_comp_2 *= (1 + momentum / 100)
        weight_comp_2[weight_comp == 0] = 1
        # weight_comp_2 is a C length vector, that is 1 * alpha if the two songs are from different clusters, 
        # and 1 if the songs are from the same cluster
        # Weights should be increased if we want to penalize it! since weighted scores = smaller the better
        
        #print(weight_comp_2)
        weights *=  weight_comp_2 # Tune momentum / 100?
        weights /= np.sum(weights) # normalize

        # Update initial song
        if val > 5:
            initial_song = other_song

        # Update song_weights
        # increase prob of landing in that cluster based on the momentum 
        song_weights[clusters == clusters[initial_song, :]] *= (1 - momentum / 100)
        # normalize
        song_weights = song_weights / np.sum(song_weights, axis = 0).reshape(1, -1)

        # Update initial_diff
        initial_diff *= (1 - momentum / 100)
        initial_diff = min(initial_diff, 100)

    # Now, weighted_songs is... weighted. 
    # Scale along axis 1
    # weighted_songs /= np.sum(weighted_songs, axis = 0)
    # # Sum across axis 1 to get scores
    # weighted_songs_sum = np.sum(weighted_songs, axis = 1)
    # # weight it by which cluster we care more
    # weighted_songs_sum *= weights
    cluster_diffs = clusters - clusters[initial_song, :] # N x C 
    cluster_diffs_ones = np.ones(clusters.shape)
    cluster_diffs_ones[cluster_diffs == 0] = 0
    weighted_songs = np.mean(cluster_diffs_ones * weights.reshape(1, -1), axis = 1)
    
    out_vec = weighted_songs <= np.percentile(weighted_songs, k / N * 100, interpolation = 'nearest')
    #print(weights)
    #print(song_weights)
    #print(weighted_songs[out_vec])
    # for ind in list(np.nonzero(out_vec))[0]:
    #     print(song_names[ind], artist_names[ind])

    weighted_songs_2 = np.mean(song_weights * weights.reshape(1, -1), axis = 1)
    out_vec = weighted_songs_2 <= np.percentile(weighted_songs_2, k / N * 100, interpolation = 'nearest')
    for ind in list(np.nonzero(out_vec))[0]:
        print(f'{song_names[ind]} by {artist_names[ind]}. Score: {weighted_songs_2[ind]}')






In [3]:
## Download data
tracks = pd.read_csv('tracks_qx.csv')
clusters = np.load('song_clusters_qx.npy').T
tracks2 = tracks[['name','artists2']]
tracks3 = tracks2.drop_duplicates()
clusters = clusters[list(tracks3.index), :]
artist_names = list(tracks3['artists2'])
track_names = list(tracks3['name'])
tracks3 = tracks3.reset_index()

In [4]:
tracks3

,index,name,artists2
0,0,Feelings,lauv
1,1,In Your Eyes,the weeknd
2,2,ANGOSTURA,keshi
3,3,Pretty Places,aly & aj
4,4,When You're Gone,shawn mendes
...,...,...,...
723,741,The Mother We Share,chvrches
724,742,Dirty Paws,of monsters and men
725,743,Best Day Of My Life,american authors
726,744,Do I Wanna Know?,arctic monkeys


In [37]:
print([ i for i in tracks.artists2.unique()])

['will young', 'bruce cockburn', 'tori amos', 'rufio', 'bride', 'the call', 'westlife', 'resurrection band', 'the highwaymen', 'the stone roses', 'johnny winter', 'shawn colvin', 'the judds', 'carter the unstoppable sex machine', 'badly drawn boy', 'lambchop', 'death cab for cutie', 'jars of clay', 'natalie imbruglia', 'blue öyster cult', 'asleep at the wheel', 'george enescu', 'franz liszt', 'sergei rachmaninoff', 'johann sebastian bach', 'ottorino respighi', 'the kentucky headhunters', '7 year bitch', 'heavens to betsy', 'bikini kill', 'melvins', 'incubus', 'the charlie daniels band', 'alan parsons', 'black sabbath', 'marilyn manson', 'papa roach', 'linkin park', 'disturbed', 'mudvayne', 'drowning pool', 'otep', 'nonpoint', 'hatebreed', 'american head charge', 'iggy azalea', 'cranes', 'toad the wet sprocket', 'the raveonettes', 'endo', 'pietro locatelli', 'the psychedelic furs', 'matthew dear', 'basement jaxx', 'boards of canada', 'ludwig van beethoven', '*nsync', 'benjamin britten',

In [5]:
## Tools to check if your song is in the track
artist = "ed sheeran"
tracks3[tracks3['artists2'] == artist]


,index,name,artists2
168,172,The A Team,ed sheeran
367,375,One,ed sheeran
368,376,I'm a Mess,ed sheeran
383,391,Photograph,ed sheeran
386,394,Thinking out Loud,ed sheeran
448,457,Castle on the Hill,ed sheeran
490,499,Perfect,ed sheeran


In [6]:
song = "The A Team"
tracks3[tracks3['name'] == song]

,index,name,artists2
168,172,The A Team,ed sheeran


0             Love The One You're With
1                            Your Game
2                             Stronger
3                      Leave Right Now
4         Love Is A Matter Of Distance
                      ...             
230174         Short Red Silk Lingerie
230175                   Warm December
230176                  In The Morning
230177                      Blue Skies
124469                            Hero
Name: name, Length: 195776, dtype: object

In [16]:
#selection_options = np.arange(5)
#cluster = np.random.choice(selection_options, 1000).reshape((100, 10))
num_clusters = clusters.shape[1]
weight = np.ones(num_clusters) / num_clusters
initial_song = 168
top_k(clusters, weight, initial_song, track_names, artist_names, k = 10, initial_diff = 15, tol = 1e-1, L = 5)

Question Number: 1, Percentile: 15, M: 0, value selected: 3
The A Team ed sheeran , When He's Done tei shi
Question Number: 2, Percentile: 13.125, M: 12.5, value selected: 1
The A Team ed sheeran , Everybody Hurts r.e.m.
Question Number: 3, Percentile: 9.3515625, M: 28.75, value selected: 2
The A Team ed sheeran , You're Somebody Else flora cash
Question Number: 4, Percentile: 6.3707519531250005, M: 31.875, value selected: 1
The A Team ed sheeran , Rocket Man (I Think It's Going To Be A Long, Long Time) elton john
Question Number: 5, Percentile: 3.9219941711425785, M: 38.4375, value selected: 7
The A Team ed sheeran , Photograph ed sheeran
Question Number: 6, Percentile: 3.4623854792118074, M: 11.71875, value selected: 3
Photograph ed sheeran , This Town niall horan
Question Number: 7, Percentile: 2.8267131451377647, M: 18.359375, value selected: 2
Photograph ed sheeran , 泡沫 g.e.m.
Question Number: 8, Percentile: 2.0725549114935875, M: 26.6796875, value selected: 1
Photograph ed sheera